In [9]:
from haversine import haversine
import pandas as pd
import os

In [10]:
DF1_FILENAME = '6.Candidate groups.csv'
DF1_FILEPATH = os.path.join(os.path.dirname(os.getcwd()),'data','dataset',DF1_FILENAME)

DF2_FILENAME = '기존충전소위경도정보.csv'
DF2_FILEPATH = os.path.join(os.path.dirname(os.getcwd()),'data','dataset',DF2_FILENAME)

candidate_groups = pd.read_csv(DF1_FILEPATH, index_col=0)
existing_charging_station = pd.read_csv(DF2_FILEPATH, index_col=0)

In [11]:
candidate_groups['existing'] = 0
candidate_groups.rename(columns={'cluster':'cluster_이름'},inplace=True)
candidate_groups

,cluster_이름,xcrds,ycrds,existing
0,1000,127.191385,37.235027,0
1,1001,127.295784,37.188830,0
2,10039,127.110995,37.301494,0
3,10040,127.203855,37.144468,0
4,1005,127.258105,37.347407,0
...,...,...,...,...
5228,994,127.254190,37.248248,0
5229,995,127.204090,37.302216,0
5230,997,127.317470,37.192700,0
5231,9986,127.086575,37.330033,0


In [12]:
existing_charging_station['existing'] = 1
existing_charging_station.rename(columns={'이름':'cluster_이름','위도':'ycrds','경도':'xcrds'},inplace=True)
existing_charging_station = existing_charging_station.reindex(['cluster_이름', 'xcrds', 'ycrds', 'existing'], axis = 1)
existing_charging_station

,cluster_이름,xcrds,ycrds,existing
0,신동백 두산위브더제니스,127.130787,37.306879,1
1,죽전역 솔하임,127.114735,37.328545,1
2,광교상현 솔하임,127.074668,37.302515,1
3,성복역 롯데캐슬 클라시엘(파크나인2차),127.061733,37.319601,1
4,까뮤이스테이트,127.109361,37.335116,1
...,...,...,...,...
1256,나인무인텔,127.320402,37.208291,1
1257,명가,127.232383,37.277380,1
1258,금화마을대우현대아파트,127.110847,37.264701,1
1259,플랫폼시티아이브오피스텔,127.110283,37.298363,1


### 거리 탐색

In [13]:
from turtle import distance

def get_min_node_idx(main_idx, main, compare_points):
    '''
    main을 기준으로 compare_porints들의 거리를 비교한 후 가장 가까운 위치의
    node행의 index를 반환
    '''
    distance_compare_list = [] # 각 main과 비교한 i 위치 거리
    under_list = [] # 특정 거리 내의 노드 Index
    hbl = (main["ycrds"][main_idx], main["xcrds"][main_idx])
    
    for i in range(compare_points.shape[0]):
        hcl = (compare_points["ycrds"][i], compare_points["xcrds"][i])
        result = haversine(hbl, hcl, unit='km')
        distance_compare_list.append(result)
        
    if min(distance_compare_list) < 1:
        under_list = [idx for idx, val in enumerate(distance_compare_list) if val < 1]
    return under_list

In [14]:
distance_pair = set()

for i in range(existing_charging_station.shape[0]):
    distance_pair.update(get_min_node_idx(i, existing_charging_station, candidate_groups))

### 비교한 값에 따라 기존에 설치되어 있는 충전소에서 1KM내에 있는 후보군 위치를 제거

In [15]:
eliminate_list = list(distance_pair) # 존재하는 충전소와 가까워 제거할 후보군 위치

fin_location = candidate_groups.copy()
fin_location.drop(eliminate_list, inplace=True)

In [16]:
fin_location = pd.concat([fin_location, existing_charging_station])
fin_location = fin_location.reset_index(drop=True)

In [17]:
fin_location

,cluster_이름,xcrds,ycrds,existing
0,1001,127.295784,37.188830,0
1,1013,127.223315,37.135049,0
2,1014,127.213345,37.315307,0
3,1017,127.227875,37.354329,0
4,10206,127.419215,37.135348,0
...,...,...,...,...
2164,나인무인텔,127.320402,37.208291,1
2165,명가,127.232383,37.277380,1
2166,금화마을대우현대아파트,127.110847,37.264701,1
2167,플랫폼시티아이브오피스텔,127.110283,37.298363,1


In [18]:
cust_filter_df = pd.read_csv('../data/dataset/8.cluster_with_count_cust.csv', index_col=0)

In [19]:
cust_filter_df

,cluster,count_cust
0,0,1744.761
1,1,435.581
2,2,80.122
3,3,640.832
4,4,407.512
...,...,...
51695,51795,307.560
51696,51796,1800.723
51697,51797,16.191
51698,51798,58.353


In [24]:
demand_satisfied_clusters_sr = cust_filter_df[cust_filter_df['count_cust'] > 62.5]['cluster']

In [25]:
fin_location['demand_satisfied'] = fin_location['cluster_이름'].isin(demand_satisfied_clusters_sr)

In [28]:
fin_location['demand_satisfied'] = fin_location['demand_satisfied'].astype('int')

In [31]:
fin_location[fin_location['demand_satisfied'] == 1]

,cluster_이름,xcrds,ycrds,existing,demand_satisfied
3,1017,127.227875,37.354329,0,1
6,1027,127.143215,37.168865,0,1
7,10309,127.193227,37.336440,0,1
8,10371,127.133110,37.161848,0,1
9,10451,127.225800,37.147452,0,1
...,...,...,...,...,...
898,887,127.140208,37.148798,0,1
902,931,127.317915,37.239798,0,1
903,963,127.368197,37.177893,0,1
906,99,127.252040,37.302776,0,1


In [30]:
fin_location.to_csv(f'../data/dataset/7.fin_location.csv')